In [2]:
import numpy as np
import pandas as pd
from numpy.random import rand

# Scikit Imports
from sklearn.model_selection import train_test_split

# pandas display data frames as tables
from IPython.display import display, HTML

from keras import regularizers, losses
from keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras import layers
from keras.layers import Dropout, GlobalAveragePooling2D, BatchNormalization, Activation,Dense, Conv2D
from keras.models import Sequential, load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.applications.inception_v3 import InceptionV3

from tqdm import tqdm

In [3]:
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))
print(sess)

Num GPUs Available:  1
Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5



### Dog Breed Identification/Classification

We have a CV problem set, we build different models and apply TL technique to improve accuracy. A model is build from scratch (CNN) and another model is build with TL through Inception model by Google.

In [4]:
from google.colab import drive
drive.mount('/content/drive/')

import os
os.chdir("/content/drive/My Drive/Colab Notebooks/Tensorflow revision/Transfer Learning/dog-breed-identification")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [5]:
train_folder = 'train/'
# Not taking test data into consideration, files to large to process on google colab's RAM lol
# test_folder = 'test/'

data_labels = pd.read_csv('labels.csv')
data_labels.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [6]:
data_labels["breed"].value_counts()

scottish_deerhound      126
maltese_dog             117
afghan_hound            116
entlebucher             115
bernese_mountain_dog    114
                       ... 
golden_retriever         67
brabancon_griffon        67
komondor                 67
eskimo_dog               66
briard                   66
Name: breed, Length: 120, dtype: int64

In [7]:
data_labels.shape

(10222, 2)

In [8]:
# Not wokring witht he whole data
new_data = pd.DataFrame()
temp = pd.DataFrame()
for breed in list(data_labels["breed"].value_counts().keys())[:40]:
  temp = data_labels[data_labels["breed"] == breed].iloc[:60, :]
  new_data = pd.concat([new_data, temp], axis=0)

In [9]:
new_data.shape

(2400, 2)

In [10]:
labels_ohe_names = pd.get_dummies(new_data.iloc[:, 1:], sparse=True)
labels_ohe = np.asarray(labels_ohe_names)
print(labels_ohe.shape)

(2400, 40)


In [11]:
new_data['image_path'] = new_data.apply( lambda row: (train_folder + row["id"] + ".jpg" ), axis=1)
new_data.head()

,id,breed,image_path
9,0042188c895a2f14ef64a918ed9c7b64,scottish_deerhound,train/0042188c895a2f14ef64a918ed9c7b64.jpg
79,01e787576c003930f96c966f9c3e1d44,scottish_deerhound,train/01e787576c003930f96c966f9c3e1d44.jpg
161,03dc61595ad9dbf49e3998cf586ca8cb,scottish_deerhound,train/03dc61595ad9dbf49e3998cf586ca8cb.jpg
233,056b535b441278e83839984f1b1da0a6,scottish_deerhound,train/056b535b441278e83839984f1b1da0a6.jpg
235,05719b998e57a11b863a322ecc7652a5,scottish_deerhound,train/05719b998e57a11b863a322ecc7652a5.jpg


### Preparing Image data

In [ ]:
train_data = np.array([img_to_array(
                                    load_img(img, target_size=(299, 299))
                                    ) for img in tqdm(new_data['image_path'].values.tolist())
                      ]).astype('float32')

train_data.shape                      

In [13]:
x_train, x_test, y_train, y_test = train_test_split(train_data, 
                                                    labels_ohe, 
                                                    test_size=0.3, 
                                                    # stratify=np.array(data_labels.iloc[:, 1:]), 
                                                    random_state=42)

In [14]:
# Validation set

x_train, x_val, y_train, y_val = train_test_split(x_train, 
                                                    y_train, 
                                                    test_size=0.15, 
                                                    # stratify=np.array(y_train), 
                                                    random_state=42)

In [13]:
# y_train_ohe = pd.get_dummies(y_train.reset_index(drop=True)).as_matrix()
# y_val_ohe = pd.get_dummies(y_val.reset_index(drop=True)).as_matrix()
# y_test_ohe = pd.get_dummies(y_test.reset_index(drop=True)).as_matrix()

# y_train_ohe.shape, y_test_ohe.shape, y_val_ohe.shape

In [ ]:
# Building a basic CNN model

base_model = Sequential(
    [
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(299, 299, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(40)
    ]    
)

In [16]:
base_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 297, 297, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 148, 148, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 146, 146, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 73, 73, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 71, 71, 64)        36928     
                                                                 
 flatten (Flatten)           (None, 322624)            0

base

In [16]:
base_model.compile(
    loss = losses.CategoricalCrossentropy(from_logits=True),
    optimizer = Adam(learning_rate=3e-4),
    metrics=["accuracy"]
)

history = base_model.fit(x_train, y_train, batch_size=32, epochs=5, 
                verbose=True, validation_data=(x_val, y_val))

base_model.evaluate(x_test, y_test, batch_size=32, verbose=True)

Epoch 1/5
47/47 [==============================] - 20s 116ms/step - loss: 81.0141 - accuracy: 0.0121 - val_loss: 4.7938 - val_accuracy: 0.0000e+00
Epoch 2/5
47/47 [==============================] - 4s 89ms/step - loss: 4.2398 - accuracy: 0.1701 - val_loss: 5.1297 - val_accuracy: 0.0114
Epoch 3/5
47/47 [==============================] - 5s 103ms/step - loss: 1.3803 - accuracy: 0.7673 - val_loss: 6.2430 - val_accuracy: 0.0076
Epoch 4/5
47/47 [==============================] - 4s 90ms/step - loss: 0.0965 - accuracy: 0.9879 - val_loss: 9.4479 - val_accuracy: 0.0266
Epoch 5/5
24/24 [==============================] - 2s 51ms/step - loss: 8.8149 - accuracy: 0.0173


[8.814936637878418, 0.01733333244919777]

In [17]:
#From the above results, it is clear that the model's generalization capabilities are exceptionally bad, 
# this is due to two main reasons:
# 1. The whole dataset is not taken, meaning not all classes have instances in training data but the 
# last FC layer assumes that the training data has 120 classes.
# 2. The data is small, from the whole of 10220 datapoints, we only use 2500.

### Transfer Leaning

In [17]:
# We use the pre-trained model Inception v3 from google for our dog breed classification problem

base_inception = InceptionV3(weights='imagenet', include_top = False, input_shape=(299, 299, 3))

In [21]:
# Add a global spatial average pooling layer
out = base_inception.output
out = GlobalAveragePooling2D()(out)
out = Dense(512, activation='relu')(out)
# out = Dense(512, activation='relu')(out)
predictions = Dense(40, activation='softmax')(out)

In [22]:
model = Model(inputs=base_inception.input, outputs=predictions)

# only if we want to freeze layers
for layer in base_inception.layers:
    layer.trainable = False
    
# Compile 
model.compile(Adam(lr=.0001), loss='categorical_crossentropy', metrics=['accuracy']) 

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_3 (Conv2D)              (None, 149, 149, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 149, 149, 32  96         ['conv2d_3[0][0]']               
 alization)                     )                                                           

In [23]:
batch_size = 32

history = model.fit(x_train, y_train, batch_size=32, epochs=15, 
                verbose=True, validation_data=(x_val, y_val))

Epoch 1/15
45/45 [==============================] - 36s 287ms/step - loss: 27.2935 - accuracy: 0.0343 - val_loss: 9.6761 - val_accuracy: 0.0278
Epoch 2/15
45/45 [==============================] - 7s 154ms/step - loss: 5.6962 - accuracy: 0.0462 - val_loss: 4.8485 - val_accuracy: 0.0397
Epoch 3/15
45/45 [==============================] - 7s 156ms/step - loss: 3.7058 - accuracy: 0.0525 - val_loss: 3.6715 - val_accuracy: 0.0397
Epoch 4/15
45/45 [==============================] - 7s 150ms/step - loss: 3.5857 - accuracy: 0.0539 - val_loss: 3.7266 - val_accuracy: 0.0317
Epoch 5/15
45/45 [==============================] - 7s 154ms/step - loss: 3.5668 - accuracy: 0.0532 - val_loss: 3.7596 - val_accuracy: 0.0437
Epoch 6/15
45/45 [==============================] - 7s 155ms/step - loss: 3.5793 - accuracy: 0.0623 - val_loss: 3.6668 - val_accuracy: 0.0198
Epoch 7/15
45/45 [==============================] - 6s 145ms/step - loss: 3.5203 - accuracy: 0.0651 - val_loss: 3.7411 - val_accuracy: 0.0278
Epoc

In [28]:
# Even with incomplete data (due to resource limitation) we can see the model not overfitting as was the
# case for our CNN model.